# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [73]:
# Initiating modules and Cards suits, rankes and values:

import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

#Defining classes used in the gameplay
class Card:
    '''
    The Card class initiates the cards, every card has a suit, a rank and a value.
    You can print out a card, showint its suit and rank.
    '''
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return f"{self.rank} of {self.suit}"


class Deck:
    '''
    The Deck class initiates a new Deck, which you can shuffle and draw a card from. You can print out the deck, showing the cards in it in order
    '''
    def __init__(self):
        self.all_cards=[]
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit,rank))        

    def shuffle(self):
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        return self.all_cards.pop()

    def __str__(self):
        self.cardlist = ''
        num = 0
        for c in self.all_cards:
            num +=1
            self.cardlist += f"  {num}. card: {c.rank} of {c.suit}\n"
        return f'Cards in the Deck:\n{self.cardlist}'

    def __len__(self):
        return len(self.all_cards)

class Player:
    '''
    The Player class initiates a PLAYER, whith a name and with zero cards.
    The eval_hand method computes the value of cards in the hand of the player.
    
    Printing the playes shows what cards he/she holds and gives back the total value of their cards.
    '''
    def __init__(self, name, chips, hand=1):
        self.name = f'Hand {hand} of {name}'
        self.all_cards = []
        self.statistics = {"wins": 0, "losses": 0, "ties": 0, "balance": chips, "gain": 0}

    def eval_hand(self):
        
        self.value_all = 0
        self.num_aces=0
        self.status='bust'
        self.value_hand=0
        
        for c in self.all_cards:
            self.value_all += c.value
            
            if c.rank == 'Ace':
                self.num_aces += 1

        for i in range(self.num_aces,-1,-1):
            if self.value_all - i*10 == 21:
                if len(self.all_cards) == 2:
                    self.status = 'blackjack'
                    self.value_hand=22
                    break
                else:
                    self.status = '21'
                    self.value_hand=21
                    break
            if 17 <= self.value_all - i*10 < 21:
                self.status = 'stop_zone'
                self.value_hand=self.value_all - i*10
                break
            if self.value_all - i*10 < 17:
                self.status = 'in_play'
                if self.value_hand < self.value_all - i*10:
                    self.value_hand = self.value_all - i*10
 
        return self.value_all, self.num_aces, self.status, self.value_hand
    
    def __str__(self):
        
        self.card_list=''
        self.value_hand_variants=str(self.value_all)
        
        for c in self.all_cards:
            self.card_list += f"   {c.rank} of {c.suit}\n"
            
        if self.num_aces > 0:
            for i in range(self.num_aces):
                self.value_hand_variants += f' or {self.value_all - (i+1)*10}'
            
        return f'{self.name} has {len(self.all_cards)} cards:\n{self.card_list}\nThe value of card(s) is {self.value_hand_variants}' 

    def win_bet(self, bet):
        self.statistics['balance'] += bet
        self.statistics['gain'] += bet

    def lose_bet(self, bet):
        self.statistics['balance'] -= bet
        self.statistics['gain'] -= bet
        

In [82]:
#defining funcions

def ask_for_players():

    '''
    Asking how many players do we have. 
    Should provide a positive whole number between 1 and 5, the funcion checks it and asks again if it is not the case
    '''
    total = 'WRONG'
    while total.isdigit() == False:

        total=input("Please let us know how many players are going to play (1 to 5): ")
        
        if total.isdigit() == False:
            print("Please provide a positive whole number!")
        else:
            if int(total) <= 0 or int(total) > 5:
                print("Please provide a whole number between 1 and 5!")
                total="WRONG"
            
    return int(total)


def ask_for_chips():

    '''
    Asking the player how many chips he/she has. 
    Should provide a positive whole number, the funcion checks it and asks again if it is not the case
    '''
    total = 'WRONG'
    while total.isdigit()==False:

        total=input("Please let us know how many chips do you have to play with: ")
        
        if total.isdigit()==False:
            print("Please provide a positive whole number!")
        else:
            if int(total)<=0:
                print("Please provide a positive whole number!")
                total="WRONG"
            
    return int(total)

def ask_for_bet(player):

    '''
    Asking the player how many chips he/she wants to risk in the next round. 
    Should provide a positive whole number, the funcion checks it and asks again if it is not the case
    Also, should provide a bet less then the amount of his/her total chips
    '''
    bet = 'WRONG'
    
    while bet.isdigit()==False:
        bet=input(f"{player.name}, what is your bet for the next game? You have {player.statistics['balance']} chips in total) ")
        
        if bet.isdigit()==False:
            print("Please provide a positive whole number!")
        else:
            if int(bet)<=0:
                print("Please provide a positive whole number!")
                bet="WRONG"
            elif int(bet) > player.chips:
                print(f"{player.name}, you have less chips than that ({player.statistics['balance']} in total)")
                bet="WRONG"
                
    return int(bet)

def player_action(player):
    '''
    Asking the player about his next move. 
    Should provide a valid answer, the funcion checks it and asks again if it is not the case
    The player can move only until he/she reaches or exceeds 21.
    '''
    acceptable_moves=["hit", "stand"]
    move="wrong"
    
    while move.lower() not in acceptable_moves:
        move = input(f"{player.name}, what is your next move, hit or stand? Please chose (write hit or stand): " ).lower()
    
        if move not in acceptable_moves:
            print("Your choice is not valid, please provide a valid move!")
    
    return move
    
def winner(players, dealer, players_bet, final=1):
    
    dealer.eval_hand()
    
    for i in range(len(players)): 
        players[i].eval_hand()
        
    if final==0 and dealer.status == 'blackjack':
        
        print(dealer)
        
        print("The dealer has a blackjack!")
        game_on = False
        
        for i in range(num_of_players): 
            
            if players[i].status=='blackjack':
                print(f"{players[i].name} has a black jack too, there is a tie")
                players[i].statistics['ties'] += 1
                
            else: 
                print(f"{players[i].name} lost the game")
                players[i].statistics['losses'] += 1
                lose_bet(players[i],players_bet[i])
                
    elif final==0:
        game_on=True
    
    else:
        game_on=False
        for i in range(num_of_players): 
            
            if players[i].value_hand > dealer.value_hand:
                print(f"{players[i].name} won with a hand value of {players[i].value_hand}")
                players[i].statistics['wins'] += 1
                win_bet(players[i],players_bet[i])
                
            elif players[i].value_hand == dealer.value_hand:
                print(f"{players[i].name} has a tie with the dealer with a hand value of {players[i].value_hand}")
                players[i].statistics['ties'] += 1
            
            else: 
                print(f"{players[i].name} lost the game with a hand value of {players[i].value_hand}")
                players[i].statistics['losses'] += 1
                lose_bet(players[i],players_bet[i])
        
    return game_on
    

def ask_for_playon(player):
    '''
    Asking the player whether they want to play another game. 
    Should provide a valid answer, the funcion checks it and asks again if it is not the case
    '''
    
    acceptable_answers=["y", "n"]
    answer="wrong"
    
    while answer.lower() not in acceptable_answers:
        answer = input(f"{player}, do you want to play another game (Y or N): " ).lower()
    
        if answer not in acceptable_answers:
            print("Your choice is not valid, please provide a valid answer!")
    
    return answer == 'y'

In [72]:
#initiating the game
play_on = True


#Informing the players about the game
print("Wellcome to our Black Jack game!")
print("You will play against or computer dealer as in a Casino")


#initiating players and their chips
players=[]
num_of_players = ask_for_players()
for i in range(num_of_players):
    name = input("Please let us know your name: ")
    chips_total = ask_for_chips()
    players[i] = Player(name, chips_total)

dealer = Player("Dealer", 1)

while play_on:
    #initiating the Deck
    play_deck = Deck()
    play_deck.shuffle()
    
    #placing the bet and starting the game
    players_bet=[]
    for i in range(num_of_players):
        players_bet[i] = ask_for_bet(players[i])
        
    dealer_bet = 0
    
    game_on = True

    #empty the hands of players and deal the first two cards
    dealer.all_cards = []
    for i in range(num_of_players):
        players[i].all_cards = []
        
    for i in range(2):
        for i in range(num_of_players):
            players[i].all_cards.append(play_deck.deal_one())
            
        dealer.all_cards.append(play_deck.deal_one())
        
    #Show his/her cards and one of the dealer cards to the players
    for i in range(num_of_players):
        players[i].eval_hand()
        print(players[i])

    dealer.eval_hand()
    print(f"\nThe first card of the dealer is {dealer.all_cards[0]}")
    
    #Check if the game is over already
    winner(players, dealer, players_bet, 0)
    
    if game_on:
        
   

    break

Wellcome to our Black Jack game!
You will play against or computer dealer as in a Casino


KeyboardInterrupt: Interrupted by user

In [75]:
player_one=Player("Ika",100)

In [76]:
player_one.all_cards.extend([Card('Hearts','King'),Card('Hearts','Two'),Card('Hearts','Ace')])

In [77]:
player_one.eval_hand()

(23, 1, 'in_play', 13)

In [78]:
print(player_one)

Hand 1 of Ika has 3 cards:
   King of Hearts
   Two of Hearts
   Ace of Hearts

The value of card(s) is 23 or 13


In [79]:
player_one.status

'in_play'

In [114]:
new_deck=Deck()

In [115]:
new_deck.shuffle()

In [117]:
card=new_deck.deal_one()
print(card)

Five of Spades


In [158]:

num=2

range(21, num*10+22, 10)

range(21, 42, 10)

In [9]:
for i in range(1, -1, -1):
    print(i)


1
0


In [53]:
num_aces=0
value_hand=22
all_cards=[Card('Hearts','King'),Card('Hearts','Ace')]

status='bust'
        
for i in range(num_aces,-1,-1):
    if value_hand - i*10 == 21:
        if len(all_cards) == 2:
            status = 'blackjack'
            break
        else:
            status = '21'
            break
    if 17 <= value_hand - i*10 < 21:
        status = 'stop_zone'
        break
    if value_hand - i*10 < 17:
        status = 'in_play'


print(status)

bust
